# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [4]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

loan_amnt  int_rate  installment home_ownership  annual_inc  \
0    10500.0    0.1719       375.35           RENT     66000.0   
1    25000.0    0.2000       929.09       MORTGAGE    105000.0   
2    20000.0    0.2000       529.88       MORTGAGE     56000.0   
3    10000.0    0.1640       353.55           RENT     92000.0   
4    22000.0    0.1474       520.39       MORTGAGE     52000.0   

  verification_status   issue_d loan_status pymnt_plan    dti  ...  \
0     Source Verified  Mar-2019    low_risk          n  27.24  ...   
1            Verified  Mar-2019    low_risk          n  20.23  ...   
2            Verified  Mar-2019    low_risk          n  24.26  ...   
3            Verified  Mar-2019    low_risk          n  31.44  ...   
4        Not Verified  Mar-2019    low_risk          n  18.76  ...   

   pct_tl_nvr_dlq  percent_bc_gt_75  pub_rec_bankruptcies  tax_liens  \
0            85.7             100.0                   0.0        0.0   
1            91.2              50.0                   1.0        0.0   
2            66.7              50.0                   0.0        0.0   
3           100.0              50.0                   1.0        0.0   
4           100.0               0.0                   0.0        0.0   

   tot_hi_cred_lim  total_bal_ex_mort total_bc_limit  \
0          65687.0            38199.0         2000.0   
1         271427.0            60641.0        41200.0   
2          60644.0            45684.0         7500.0   
3          99506.0            68784.0        19700.0   
4         219750.0            25919.0        27600.0   

   total_il_high_credit_limit  hardship_flag  debt_settlement_flag  
0                     61987.0              N                     N  
1                     49197.0              N                     N  
2                     43144.0              N                     N  
3                     76506.0              N                     N  
4                     20000.0              N                     N  

[5 rows x 86 columns]

# Split the Data into Training and Testing

In [5]:
# Create our features
X = pd.get_dummies(df.drop(columns='loan_status'))


# Create our target
y = y = df['loan_status']

In [6]:
# Confirm/Display
X.describe()

loan_amnt      int_rate   installment    annual_inc           dti  \
count  68817.000000  68817.000000  68817.000000  6.881700e+04  68817.000000   
mean   16677.594562      0.127718    480.652863  8.821371e+04     21.778153   
std    10277.348590      0.048130    288.062432  1.155800e+05     20.199244   
min     1000.000000      0.060000     30.890000  4.000000e+01      0.000000   
25%     9000.000000      0.088100    265.730000  5.000000e+04     13.890000   
50%    15000.000000      0.118000    404.560000  7.300000e+04     19.760000   
75%    24000.000000      0.155700    648.100000  1.040000e+05     26.660000   
max    40000.000000      0.308400   1676.230000  8.797500e+06    999.000000   

        delinq_2yrs  inq_last_6mths      open_acc       pub_rec  \
count  68817.000000    68817.000000  68817.000000  68817.000000   
mean       0.217766        0.497697     12.587340      0.126030   
std        0.718367        0.758122      6.022869      0.336797   
min        0.000000        0.000000      2.000000      0.000000   
25%        0.000000        0.000000      8.000000      0.000000   
50%        0.000000        0.000000     11.000000      0.000000   
75%        0.000000        1.000000     16.000000      0.000000   
max       18.000000        5.000000     72.000000      4.000000   

           revol_bal  ...  issue_d_Mar-2019  pymnt_plan_n  \
count   68817.000000  ...      68817.000000       68817.0   
mean    17604.142828  ...          0.177238           1.0   
std     21835.880400  ...          0.381873           0.0   
min         0.000000  ...          0.000000           1.0   
25%      6293.000000  ...          0.000000           1.0   
50%     12068.000000  ...          0.000000           1.0   
75%     21735.000000  ...          0.000000           1.0   
max    587191.000000  ...          1.000000           1.0   

       initial_list_status_f  initial_list_status_w  next_pymnt_d_Apr-2019  \
count           68817.000000           68817.000000           68817.000000   
mean                0.123879               0.876121               0.383161   
std                 0.329446               0.329446               0.486161   
min                 0.000000               0.000000               0.000000   
25%                 0.000000               1.000000               0.000000   
50%                 0.000000               1.000000               0.000000   
75%                 0.000000               1.000000               1.000000   
max                 1.000000               1.000000               1.000000   

       next_pymnt_d_May-2019  application_type_Individual  \
count           68817.000000                 68817.000000   
mean                0.616839                     0.860340   
std                 0.486161                     0.346637   
min                 0.000000                     0.000000   
25%                 0.000000                     1.000000   
50%                 1.000000                     1.000000   
75%                 1.000000                     1.000000   
max                 1.000000                     1.000000   

       application_type_Joint App  hardship_flag_N  debt_settlement_flag_N  
count                68817.000000          68817.0                 68817.0  
mean                     0.139660              1.0                     1.0  
std                      0.346637              0.0                     0.0  
min                      0.000000              1.0                     1.0  
25%                      0.000000              1.0                     1.0  
50%                      0.000000              1.0                     1.0  
75%                      0.000000              1.0                     1.0  
max                      1.000000              1.0                     1.0  

[8 rows x 95 columns]

In [7]:
# Confirm/Display
y.describe()

count        68817
unique           2
top       low_risk
freq         68470
Name: loan_status, dtype: object

In [8]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1, stratify=y)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [10]:
# Resample the training data with the RandomOversampler
# Importing the Dependencies
from imblearn.over_sampling import RandomOverSampler 

# Set the sampling state
ros = RandomOverSampler(random_state=1)

# Sample and fit 
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

# Display
Counter(y_resampled)

Counter({'low_risk': 51352, 'high_risk': 51352})

In [11]:
# Train the Logistic Regression model using the resampled data
# Importing the Dependencies

from sklearn.linear_model import LogisticRegression 

# Building the Model 
model = LogisticRegression(solver='lbfgs', random_state =1)

# Fitting the Model 
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [12]:
# Calculated the balanced accuracy score
# Importing the Dependencies

from sklearn.metrics import balanced_accuracy_score 

# Calculate 
y_pred = model.predict(X_test) 

balanced_accuracy_score(y_test, y_pred)

0.6398437216722869

In [13]:
# Display the confusion matrix
# Importing the Dependencies

from sklearn.metrics import confusion_matrix

# Building the Matrix 
confusion_matrix(y_test, y_pred) 

array([[   51,    36],
       [ 5247, 11871]])

In [14]:
# Print the imbalanced classification report
# Importing the Dependencies

from imblearn.metrics import classification_report_imbalanced 
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.59      0.69      0.02      0.64      0.40        87
   low_risk       1.00      0.69      0.59      0.82      0.64      0.41     17118

avg / total       0.99      0.69      0.59      0.81      0.64      0.41     17205



### SMOTE Oversampling

In [16]:
# Resample the training data with SMOTE

from imblearn.over_sampling import SMOTE 

X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(X_train, y_train)

In [17]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [18]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6216172933512213

In [19]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   51,    36],
       [ 5871, 11247]])

In [20]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.59      0.66      0.02      0.62      0.38        87
   low_risk       1.00      0.66      0.59      0.79      0.62      0.39     17118

avg / total       0.99      0.66      0.59      0.79      0.62      0.39     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [21]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
X_train

loan_amnt  int_rate  installment  annual_inc    dti  delinq_2yrs  \
39692    10000.0    0.1131       328.86     49000.0  24.54          0.0   
37830    21000.0    0.1072       684.74     57400.0  34.52          0.0   
64531    35000.0    0.0881      1109.90    180000.0   3.01          0.0   
28730    20000.0    0.0819       628.49     80000.0  26.51          0.0   
35102    12600.0    0.1356       427.96     71000.0  22.38          1.0   
...          ...       ...          ...         ...    ...          ...   
18872    35000.0    0.0881      1109.90    400000.0  13.79          0.0   
6415     16800.0    0.0881       532.76     31000.0  20.48          0.0   
12632    25000.0    0.1102       818.71     90000.0  15.27          1.0   
24022    14000.0    0.1072       456.49    180000.0  12.35          2.0   
44756    15000.0    0.0702       463.30     89000.0  13.30          0.0   

       inq_last_6mths  open_acc  pub_rec  revol_bal  ...  issue_d_Mar-2019  \
39692             0.0      10.0      1.0     6580.0  ...                 0   
37830             0.0      12.0      0.0    18688.0  ...                 0   
64531             2.0       7.0      0.0      836.0  ...                 0   
28730             0.0      15.0      0.0     2615.0  ...                 0   
35102             2.0       7.0      0.0     9073.0  ...                 0   
...               ...       ...      ...        ...  ...               ...   
18872             1.0      17.0      1.0     7810.0  ...                 0   
6415              0.0       3.0      0.0     1867.0  ...                 1   
12632             0.0      18.0      0.0    26106.0  ...                 0   
24022             0.0      10.0      0.0     3601.0  ...                 0   
44756             0.0       7.0      0.0     2027.0  ...                 0   

       pymnt_plan_n  initial_list_status_f  initial_list_status_w  \
39692             1                      0                      1   
37830             1                      0                      1   
64531             1                      0                      1   
28730             1                      0                      1   
35102             1                      0                      1   
...             ...                    ...                    ...   
18872             1                      0                      1   
6415              1                      0                      1   
12632             1                      0                      1   
24022             1                      0                      1   
44756             1                      0                      1   

       next_pymnt_d_Apr-2019  next_pymnt_d_May-2019  \
39692                      0                      1   
37830                      0                      1   
64531                      0                      1   
28730                      0                      1   
35102                      0                      1   
...                      ...                    ...   
18872                      1                      0   
6415                       0                      1   
12632                      1                      0   
24022                      0                      1   
44756                      1                      0   

       application_type_Individual  application_type_Joint App  \
39692                            1                           0   
37830                            1                           0   
64531                            1                           0   
28730                            1                           0   
35102                            1                           0   
...                            ...                         ...   
18872                            1                           0   
6415                             1                           0   
12632                            1                           0   
24022              

In [22]:
X_train.isna().sum().sort_values()

loan_amnt                 0
pct_tl_nvr_dlq            0
num_tl_op_past_12m        0
num_tl_90g_dpd_24m        0
num_tl_30dpd              0
                         ..
open_act_il               0
open_acc_6m               0
tot_cur_bal               0
open_rv_24m               0
debt_settlement_flag_N    0
Length: 95, dtype: int64

In [23]:
#need to explicitly cast as float64 due to error in ClusterCentroids (worked with Instructor for resolution on issue)
X_train=X_train.astype('float64')

In [24]:
import imblearn

imblearn.__version__

'0.10.1'

In [25]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
# Importing the Dependencies

from collections import Counter 
from imblearn.under_sampling import ClusterCentroids 

cc = ClusterCentroids(random_state=1)

X_resampled, y_resampled = cc.fit_resample(X_train, y_train)


#before resampling
Counter(y_resampled)

Counter({'high_risk': 260, 'low_risk': 260})

In [26]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=78)
model.fit(X_resampled,y_resampled)

LogisticRegression(random_state=78)

In [27]:
# Calculated the balanced accuracy score
#Import balanced_accuracy_score 
from sklearn.metrics import balanced_accuracy_score 

# Predicting the Y values 
y_pred = model.predict(X_test)

# Calculate the balanced_accuracy_score 
balanced_accuracy_score(y_test, y_pred)

0.5333113762081455

In [28]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  52,   35],
       [9091, 8027]])

In [29]:
# Print the Imbalanced Classification Report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.60      0.47      0.01      0.53      0.28        87
   low_risk       1.00      0.47      0.60      0.64      0.53      0.28     17118

avg / total       0.99      0.47      0.60      0.63      0.53      0.28     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [30]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
# Importing the Dependencies

from imblearn.combine import SMOTEENN

# Sample
smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X,y)

In [31]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [32]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6357786318898034

In [33]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  60,   27],
       [7157, 9961]])

In [34]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.69      0.58      0.02      0.63      0.41        87
   low_risk       1.00      0.58      0.69      0.73      0.63      0.40     17118

avg / total       0.99      0.58      0.69      0.73      0.63      0.40     17205

